In [77]:
import numpy as np
import pandas as pd
import pyBigWig
import pyranges as pr
import lotron2

In [78]:
# import argparse

# parser = argparse.ArgumentParser(description='Find candidate peaks')

# parser.add_argument('file', help='bigwig file')
# parser.add_argument('-t', '--threshold-list', nargs='*', help='values multiplied by background to set threshold for candidate peaks| DEFAULT: 2 4 6', default=[2, 4, 6])
# parser.add_argument('-b', '--background_list', nargs='*', help='values to set size of region for used for background coverage| DEFAULT: 10000 100000 1000000', default=[10000, 100000, 1000000])
# parser.add_argument('-w', '--window-list', nargs='*', help='window size in basepairs used to smooth bigwig file| DEFAULT: 200 400 600', default=[200, 400, 600])
# parser.add_argument('-c', '--threshold-cumulative', type=int, help='number of different enrichment tests passed by region to be called a candidate peak; max = threshold-list count * background-list count * window-list count| DEFAULT: 9', default=9)
# parser.add_argument('-n', '--min', type=int, help='minimum peak size| DEFAULT: 50', default=50)
# parser.add_argument('-x', '--max', type=int, help='maximum peak size| DEFAULT: 500', default=500)
# parser.add_argument('-f', '--folder', type=str, help='folder to write results to| DEFAULT: ./', default='./')

# args = parser.parse_args()

# bw_file = args.file
# threshold_list = args.threshold_list
# threshold_list = [float(i) for i in threshold_list]
# background_list = args.background_list
# background_list = [int(i) for i in background_list]
# window_list = args.window_list
# window_list = [int(i) for i in window_list]
# threshold_cumulative_init = args.threshold_cumulative
# threshold_cumulative_init = int(threshold_cumulative_init)
# min_size = args.min
# min_size = int(min_size)
# max_size = args.max
# max_size = int(max_size)
# folder = args.folder

In [79]:
eryth_atac = lotron2.BigwigData('/project/hugheslab/datashare/lhentges/biobank/atac/Don001_ATAC_d13.hg38.bw')

In [80]:
bw_file = '/project/hugheslab/datashare/lhentges/biobank/atac/Don001_ATAC_d13.hg38.bw'
threshold_list = [2, 4, 6]
background_list = [10**4, 10**5, 10**6]
window_list = [200, 400, 600]
threshold_cumulative_init = 9
min_size = 50
max_size = 500
folder = '/home/l/lhentges/code/repos/LoTron2/'

In [81]:
chrom_list = []
pybw_object = pyBigWig.open(bw_file)
for chrom in pybw_object.chroms():
    if (not chrom.startswith('chrUn')) and ('_' not in chrom) and (chrom != 'chrM') and (chrom != 'chrEBV'):
        chrom_list.append(chrom)
pybw_object.close()

In [82]:
bw_data = lotron2.BigwigData(bw_file)

In [83]:
total_df = pd.DataFrame(columns=['Chromosome', 'Start', 'End'])

In [84]:
threshold_limit = len(background_list)*len(window_list)*len(threshold_list)

for chrom in chrom_list:
    print(chrom)
    chrom = 'chr22'
    print(chrom)
    
    threshold_cumulative = threshold_cumulative_init
    chrom_coverage_array, chrom_stats_dict = bw_data.get_chrom_info_make_coverage_map(chrom, return_chrom_stats_dict=True)
    background_global_min = chrom_stats_dict['chrom_mean']
    enriched_regions, threshold_array = lotron2.find_enriched_regions_param_grid(chrom_coverage_array, background_list, window_list, threshold_list, threshold_cumulative, background_global_min=background_global_min, return_threshold_array=True)


    enriched_regions_df = pd.DataFrame(enriched_regions, columns=['Start', 'End'])
    enriched_regions_df['size'] = enriched_regions_df.End - enriched_regions_df.Start
    enriched_regions_df = enriched_regions_df.drop(enriched_regions_df[enriched_regions_df['size']<min_size].index)
    enriched_regions_df['Chromosome'] = chrom

    solved_df = enriched_regions_df[enriched_regions_df['size']<=max_size].copy()
    unsolved_df = enriched_regions_df[enriched_regions_df['size']>max_size].copy()

    unsolved_df = unsolved_df[['Chromosome', 'Start', 'End']]

    while (not unsolved_df.empty) and (threshold_cumulative < threshold_limit-1):

        # sort unsolved_df by Start
        unsolved_df = unsolved_df.sort_values(['Chromosome', 'Start'])
        unsolved_df.reset_index(drop=True, inplace=True)
        unsolved_df['unsolved_idx'] = unsolved_df.index
        

        # rerun test with higher threshold
        threshold_cumulative += 1
        enriched_regions_higher_thresh = lotron2.find_enriched_regions(threshold_array, threshold_cumulative)
        enriched_regions_higher_thresh_df = pd.DataFrame(enriched_regions_higher_thresh, columns=['Start', 'End'])
        enriched_regions_higher_thresh_df['Chromosome'] = chrom


        # use pyranges to find overlaps between unsolved regions and enriched regions at a higher threshold
        unsolved_pr = pr.PyRanges(unsolved_df)
        enriched_regions_higher_thresh_pr = pr.PyRanges(enriched_regions_higher_thresh_df)
        prs_for_overlap_dict = {k: v for k, v in zip(['enriched initial', 'enriched higher thresh'], [unsolved_pr, enriched_regions_higher_thresh_pr])}
        overlap_counts_df = pr.count_overlaps(prs_for_overlap_dict).as_df()
        # overlap_counts_df['overlap_counts_df_idx'] = overlap_counts_df.index
        overlap_counts_df['size'] = overlap_counts_df.End - overlap_counts_df.Start


        # find regions that are enriched initially, but not enriched at any coordinates with higher threshold
        single_enriched_df = pd.merge(unsolved_df, overlap_counts_df[(overlap_counts_df['enriched initial'] == 1) & (overlap_counts_df['enriched higher thresh'] == 0)], on=['Chromosome', 'Start', 'End'], how='inner')
        if not single_enriched_df.empty:
            solved_df = pd.concat([solved_df, single_enriched_df])
            unsolved_df.drop(single_enriched_df['unsolved_idx'], inplace=True)


        # find regions that are both enriched initially and enriched at coordinates with higher threshold
        double_enriched_df = overlap_counts_df[(overlap_counts_df['enriched initial'] == 1) & (overlap_counts_df['enriched higher thresh'] == 1)]
        double_enriched_df = pd.merge_asof(double_enriched_df, unsolved_df, on='Start', by='Chromosome', suffixes=('_double_enriched', '_unsolved'))
        double_enriched_df['below_max_size'] = double_enriched_df['size'] < max_size


        solved_df = pd.concat([solved_df, double_enriched_df[double_enriched_df['below_max_size']]])
        unsolved_df.drop(double_enriched_df[double_enriched_df['below_max_size']]['unsolved_idx'].unique(), inplace=True)
        unsolved_df = unsolved_df[['Chromosome', 'Start', 'End']]
        

    solved_df = pd.concat([solved_df, unsolved_df])
    solved_df = solved_df[['Chromosome', 'Start', 'End']]
    total_df = pd.concat([total_df, solved_df])
    break

chr1
chr22


In [85]:
total_df = total_df.sort_values(['Chromosome', 'Start'])
total_df.to_csv(folder+'candidate_peak_beta.bed', index=False, sep='\t', header=False)

In [86]:
total_df

,Chromosome,Start,End
0,chr22,10536922,10537060.0
1,chr22,10537087,10537153.0
2,chr22,10544673,10544807.0
3,chr22,10597446,10597507.0
5,chr22,10701964,10702077.0
...,...,...,...
1021,chr22,50686272,50686358.0
41,chr22,50783460,NaN
194,chr22,50800716,NaN
1026,chr22,50806993,50807336.0
